In [188]:
from dash import Dash, html, dcc, Output, Input, dash_table
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [2]:
data = pd.read_csv('../Data/needed_food_data.csv')

In [258]:
app = Dash(__name__)

table_header_style = {
    'fontWeight': 'bold',  
    'textShadow': '2px 2px 4px rgba(0,0,0,0.2)'
}

app.layout = html.Div(style={'padding': '70px', 'paddingTop': '0px'}, children=[
    html.H3(children='Data Table', style={'textAlign': 'center', 'fontFamily': 'sans-serif'}),
    dcc.Dropdown(
        id='column',
        options=[{'label': column, 'value': column} for column in data.columns],
        value=data.columns,
        multi=True,
    ),
    dcc.Markdown('##'),
    dash_table.DataTable(
        id='data-table',
        columns=[
            {'name': column, 'id': column} for column in data.columns
        ],
        data=data.to_dict('records'),
        page_size=10,
        style_table={
            'textShadow': '2px 2px 4px rgba(0,0,0,0.4)',
        },
        style_header={
            'fontWeight': 'bold',
            'textAlign': 'center',
            'textTransform': 'capitalize',
            'backgroundColor': 'rgba(211, 211, 211, 0.7)'
        },
        style_data={
            'textAlign': 'center',
        },
    ),
    html.Hr(),
    html.H3(children='Average Food Prices Over Time', style={'textAlign': 'center', 'fontFamily': 'sans-serif'}),
    dcc.Dropdown(
        id='crops',
        options=[{'label': crop, 'value': crop} for crop in data['commodity'].unique()],
        value=[crop for crop in data['commodity'].unique()],
        multi=True
    ),
    dcc.Graph(id='crops-data', style={'height': '600px'}),
    html.Hr(),
    dcc.Graph(
        id='map', 
        style={'width': '100%', 'height': '800px', 'margin':'0', 'padding': '0'},
        config={
            "scrollZoom": False
        }
    ),
    html.Hr(),
    html.H3(children='Total Commodities vs Market Histogram', style={'textAlign': 'center', 'paddingTop':'10px', 'fontFamily':'sans-serif'}),
    dcc.Dropdown(
        id='market',
        options=[{'label': market, 'value': market} for market in data['market'].unique()],
        value=[market for market in data['market'].unique()],
        multi=True
    ),
    dcc.Graph(id='market-data', style={'height': '600px'}),
    html.Hr(),
    html.H3(children='Commodity Pie Chart', style={'textAlign': 'center', 'paddingTop':'10px', 'fontFamily':'sans-serif'}),
    dcc.Dropdown(
        id='commodity',
        options=[{'label': commodity, 'value': commodity} for commodity in data['commodity'].unique()],
        value=[commodity for commodity in data['commodity'].unique()],
        multi=True
    ),
    dcc.Graph(id='commodity-pie', style={'height': '600px'}),
    html.Hr(),
    html.H3(children='Commodities vs Market Bar Graph', style={'textAlign': 'center', 'paddingTop':'10px', 'fontFamily':'sans-serif'}),
    dcc.Dropdown(
        id='market-commodity',
        options=[{'label': market, 'value': market} for market in data['market'].unique()],
        value=[market for market in data['market'].unique()],
        multi=True
    ),
    dcc.Graph(id='market-bar', style={'height': '600px'}),
    html.Hr(),
    html.H3(children='Average Price Trend Over Years Bar Graph', style={'textAlign': 'center', 'paddingTop':'10px', 'fontFamily':'sans-serif'}),
    dcc.Dropdown(
        id='year',
        options=[{'label': year, 'value': year} for year in data['year'].unique()],
        value=[year for year in data['year'].unique()],
        multi=True
    ),
    dcc.Graph(id='price-trend', style={'height': '600px'}),
    html.Hr(),
    html.H3(children='Average Price Trend Over Months Bar Graph', style={'textAlign': 'center', 'paddingTop':'10px', 'fontFamily':'sans-serif'}),
    dcc.Dropdown(
        id='month',
        options=[{'label': month, 'value': month} for month in data['month'].unique()],
        value=[month for month in data['month'].unique()],
        multi=True
    ),
    dcc.Graph(id='month-price-trend', style={'height': '600px'}),
    html.Hr(),
    html.H3(children='Average Food Prices Over Month', style={'textAlign': 'center', 'fontFamily': 'sans-serif'}),
    dcc.Dropdown(
        id='crops-month',
        options=[{'label': crop, 'value': crop} for crop in data['commodity'].unique()],
        value=[crop for crop in data['commodity'].unique()],
        multi=True
    ),
    dcc.Graph(id='crops-month-data', style={'height': '600px'}),
    html.Hr(),
    html.H3(children='Average Price Trend Over Markets Bar Graph', style={'textAlign': 'center', 'paddingTop':'10px', 'fontFamily':'sans-serif'}),
    dcc.Dropdown(
        id='market-trend',
        options=[{'label': market, 'value': market} for market in data['market'].unique()],
        value=[market for market in data['market'].unique()],
        multi=True
    ),
    dcc.Graph(id='market-price-trend', style={'height': '600px'}),
    html.Hr(),
    html.H3(children='Average Price Trend Over Markets Line Graph', style={'textAlign': 'center', 'paddingTop':'10px', 'fontFamily':'sans-serif'}),
    dcc.Dropdown(
        id='market-trend-line',
        options=[{'label': market, 'value': market} for market in data['market'].unique()],
        value=[market for market in data['market'].unique()],
        multi=True
    ),
    dcc.Graph(id='market-price-trend-line', style={'height': '600px'}),
    html.Hr(),
    html.H3(children='Price Distribution vs Commodity Box-plot', style={'textAlign': 'center', 'paddingTop':'10px', 'fontFamily':'sans-serif'}),
    dcc.Dropdown(
        id='crop-box',
        options=[{'label': crop, 'value': crop} for crop in data['commodity'].unique()],
        value=[crop for crop in data['commodity'].unique()],
        multi=True
    ),
    dcc.Graph(id='box-crop', style={'height': '600px'}),
    html.Hr(),

])

#################################################################################################################

@app.callback(
    [Output('data-table', 'data'),
     Output('data-table', 'columns')],
    [Input('column', 'value')]
)
def update_table(selected_columns):
    filtered_data = data[selected_columns]
    columns = [{'name': column, 'id': column} for column in selected_columns]
    return filtered_data.to_dict('records'), columns

@app.callback(
    Output('crops-data', 'figure'),
    [Input('crops', 'value')]
)

def update_graph(value):
    average_prices = data.groupby(['year', 'commodity'])['price'].mean().reset_index()
    df = average_prices[average_prices['commodity'].isin(value)]
    fig = px.line(df, x='year', y='price', color='commodity', markers='line+markers')
    fig.update_layout(
        xaxis=dict(title='Year'),
        yaxis=dict(title='Average Price'),
        xaxis_tickangle=-45,
        margin=dict(l=40, r=40, t=50, b=50),
        title = f'Average Price Trends for {" , ".join(value)}',
        title_x=0.5,
        title_y=0.96
    )
    return fig

@app.callback(
    Output('map', 'figure'),
    [Input('crops', 'value')]
)
def update_map(value):
    trace = go.Scattermapbox(
        lat=data['latitude'],
        lon=data['longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='red',
        ),
        text=data['market'],
    )
    layout = go.Layout(
        autosize=True,
        hovermode='closest',
        mapbox=dict(
            center=dict(
                lat=data['latitude'].mean(),
                lon=data['longitude'].mean()
            ),
            style='open-street-map',
            zoom=5.2,
        ),
        title='Market Locations',
        margin=dict(l=40, r=40, t=40, b=30)
    )
    return {'data': [trace], 'layout': layout}

@app.callback(
    Output('market-data', 'figure'),
    [Input('market', 'value')]
)

def update_market(value):
    df = data[data['market'].isin(value)]
    fig = px.histogram(df, x='market')
    fig.update_layout(
        yaxis=dict(title='Count (x100KG)'),
        xaxis=dict(title='Market'),
        xaxis_tickangle=-30,
        margin=dict(l=40, r=40, t=50, b=40),
        title = 'Total commodities per market',
        title_x=0.5,
        title_y=0.96
    )
    return fig

@app.callback(
    Output('commodity-pie', 'figure'),
    [Input('commodity', 'value')]
)

def commodity_pie(value):
    df = data[data['commodity'].isin(value)]
    fig = px.pie(df, names='commodity', title='Commodity Distribution')
    fig.update_traces(
        textinfo='percent+label',
        pull=[0.05, 0.05, 0.05],
        marker=dict(line=dict(color='black', width=2))
    )
    fig.update_layout(
        margin=dict(t=40, b=30),
        title_x=0.5
    )
    return fig

@app.callback(
    Output('market-bar', 'figure'),
    [Input('market-commodity', 'value')]
)
def market_bar(value):
    df = data[data['market'].isin(value)]
    grouping = df.groupby('market')['commodity'].value_counts().unstack()
    fig = px.bar(grouping, x=grouping.index, y=grouping.columns, title='Commodity Counts by Market', labels={'y': 'Count', 'index': 'Market'})
    fig.update_layout(
        yaxis=dict(title='Value'),
        xaxis=dict(title='Market'),
        xaxis_tickangle=-30,
        margin=dict(l=40, r=40, t=50, b=30),
        title_x=0.5,
        legend_title='Commodity',
        title_y=0.96
    )
    return fig

@app.callback(
    Output('price-trend', 'figure'),
    [Input('year', 'value')]
)
def price_trend(value):
    df = data[data['year'].isin(value)]
    year_commodity_avg = df.groupby(['year', 'commodity'])['price'].mean().reset_index()
    fig = px.bar(
        year_commodity_avg, 
        x='year', y='price', 
        color='commodity', 
        title='Average Price of Commodities Over the Years', labels={'year': 'Year', 'price': 'Average Price'},
        barmode='group'
    )
    fig.update_layout(
        margin=dict(l=40, r=40, t=60, b=30),
        legend_title='Commodity',
        title_x=0.5,
        title_y=0.96
    )
    return fig

@app.callback(
    Output('month-price-trend', 'figure'),
    [Input('month', 'value')]
)
def price_trend(value):
    df = data[data['month'].isin(value)]
    month_commodity_avg = df.groupby(['month', 'commodity'])['price'].mean().reset_index()
    fig = px.bar(
        month_commodity_avg, 
        x='month', y='price', 
        color='commodity', 
        title='Average Price of Commodities Over the Months', labels={'month': 'Month', 'price': 'Average Price'},
        barmode='group'
    )
    fig.update_layout(
        margin=dict(l=40, r=40, t=60, b=30),
        legend_title='Commodity',
        title_x=0.5,
        title_y=0.96
    )
    return fig

@app.callback(
    Output('crops-month-data', 'figure'),
    [Input('crops-month', 'value')]
)

def update_graph(value):
    average_prices = data.groupby(['month', 'commodity'])['price'].mean().reset_index()
    df = average_prices[average_prices['commodity'].isin(value)]
    fig = px.line(df, x='month', y='price', color='commodity', markers='line+markers')
    fig.update_layout(
        xaxis=dict(title='Month'),
        yaxis=dict(title='Average Price'),
        xaxis_tickangle=-45,
        margin=dict(l=40, r=40, t=50, b=30),
        title = f'Average Price Trends for {" , ".join(value)}',
        title_x=0.5,
        title_y=0.96
    )
    return fig

@app.callback(
    Output('market-price-trend', 'figure'),
    [Input('market-trend', 'value')]
)
def price_trend(value):
    df = data[data['market'].isin(value)]
    market_commodity_avg = data.groupby(['market', 'commodity'])['price'].mean().reset_index()
    fig = px.bar(
        market_commodity_avg, 
        x='market', y='price', 
        color='commodity', 
        title='Average Price of Commodities Over the Markets', labels={'market': 'Market', 'price': 'Average Price'},
        barmode='group'
    )
    fig.update_layout(
        xaxis_tickangle=-30,
        margin=dict(l=40, r=40, t=50, b=30),
        title_x=0.5,
        legend_title='Commodity',
        title_y=0.96
    )
    return fig

@app.callback(
    Output('market-price-trend-line', 'figure'),
    [Input('market-trend-line', 'value')]
)
def price_trend(value):
    df = data[data['market'].isin(value)]
    market_commodity_avg = data.groupby(['market', 'commodity'])['price'].mean().reset_index()
    fig = px.line(
        market_commodity_avg, 
        x='market', y='price', 
        color='commodity', 
        title='Average Price of Commodities Over the Markets', labels={'market': 'Market', 'price': 'Average Price'},
    )
    fig.update_layout(
        xaxis_tickangle=-30,
        margin=dict(l=40, r=40, t=50, b=30),
        title_x=0.5,
        legend_title='Commodity',
        title_y=0.96
    )
    return fig

@app.callback(
    Output('box-crop', 'figure'),
    [Input('crop-box', 'value')]
)
def box_dist(value):
    df = data[data['commodity'].isin(value)]
    fig = px.box(df, x='commodity', y='price', title='Price Distribution by Commodity', labels={'commodity': 'Commodity', 'price':'Price'})
    fig.update_layout(
        margin=dict(l=40, r=40, t=50, b=30),
        title_x=0.5,
        title_y=0.96
    )
    return fig

if __name__ == '__main__':
    app.run(debug=True, jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
